# AES-128 Encryption and Decryption Implementation

This project is a full implementation of the **AES-128 encryption algorithm**, built from scratch in Python to understand and demonstrate how each component of the AES cipher works. 

We have implemented the complete encryption-decryption pipeline including:

---

## Project Objectives
- Understand the **inner workings of AES-128**, including substitution, permutation, and key mixing stages.
- Implement all core AES operations **without using external crypto libraries**.
- Apply **AES-128 in CBC mode** for practical, secure message encryption and decryption.
- Explore vulnerabilities and best practices, such as **correct padding** and **key handling**.
- Investigate **stream ciphers and padding-related pitfalls**, and contrast them with block ciphers like AES.

---

## Key Features Implemented

- `SubBytes` and `SubBytes_inv`: Byte-wise substitution using the AES S-box.
- `ShiftRows` and `ShiftRows_inv`: Row-wise byte permutation.
- `MixColumns` and `MixColumns_inv`: Column-wise mixing for diffusion.
- `AddRoundKey`: Key mixing through XOR.
- **Key Expansion**: Generation of round keys from the 128-bit master key using the RCON table and S-box.
- **AES128_encrypt / AES128_decrypt**: AES encryption and decryption for a single 16-byte block.
- **AES128_CBC_encrypt / AES128_CBC_decrypt**:
  - Full CBC (Cipher Block Chaining) mode support.
  - Handles arbitrary-length plaintexts with proper padding and unpadding (e.g., PKCS#7).
- Explored **One-Time Pad vulnerabilities** when the key is shorter than the message.
- Used **known-plaintext attacks** to demonstrate XOR stream cipher weaknesses under key reuse.

---

## Learning Outcomes

Through this implementation, we gained hands-on experience with:
- The structure and operations inside a modern block cipher.
- Secure key usage and the importance of **non-reuse and correct padding**.
- CBC mode operation and its real-world application scenarios.
- Vulnerability analysis via known-plaintext and key reuse attacks.

---

## Future Scope

- Add **AES-192** and **AES-256** support.
- Implement **AES-GCM or AES-CTR** for authenticated encryption.
- Build a simple CLI or web interface for secure file/message encryption.

---

> This implementation is intended for educational purposes and to build a strong foundation in cryptographic principles — not for production use without proper review and security auditing.


---------------------------------------------------------

---------------------------------------------------------

## Section 0: Preparison
We are going to develop AES-128.
Followings are some default parameters used by AES.

In [87]:
SBOX = [
    [0x63, 0x7C, 0x77, 0x7B, 0xF2, 0x6B, 0x6F, 0xC5, 0x30, 0x01, 0x67, 0x2B, 0xFE, 0xD7, 0xAB, 0x76],
    [0xCA, 0x82, 0xC9, 0x7D, 0xFA, 0x59, 0x47, 0xF0, 0xAD, 0xD4, 0xA2, 0xAF, 0x9C, 0xA4, 0x72, 0xC0],
    [0xB7, 0xFD, 0x93, 0x26, 0x36, 0x3F, 0xF7, 0xCC, 0x34, 0xA5, 0xE5, 0xF1, 0x71, 0xD8, 0x31, 0x15],
    [0x04, 0xC7, 0x23, 0xC3, 0x18, 0x96, 0x05, 0x9A, 0x07, 0x12, 0x80, 0xE2, 0xEB, 0x27, 0xB2, 0x75],
    [0x09, 0x83, 0x2C, 0x1A, 0x1B, 0x6E, 0x5A, 0xA0, 0x52, 0x3B, 0xD6, 0xB3, 0x29, 0xE3, 0x2F, 0x84],
    [0x53, 0xD1, 0x00, 0xED, 0x20, 0xFC, 0xB1, 0x5B, 0x6A, 0xCB, 0xBE, 0x39, 0x4A, 0x4C, 0x58, 0xCF],
    [0xD0, 0xEF, 0xAA, 0xFB, 0x43, 0x4D, 0x33, 0x85, 0x45, 0xF9, 0x02, 0x7F, 0x50, 0x3C, 0x9F, 0xA8],
    [0x51, 0xA3, 0x40, 0x8F, 0x92, 0x9D, 0x38, 0xF5, 0xBC, 0xB6, 0xDA, 0x21, 0x10, 0xFF, 0xF3, 0xD2],
    [0xCD, 0x0C, 0x13, 0xEC, 0x5F, 0x97, 0x44, 0x17, 0xC4, 0xA7, 0x7E, 0x3D, 0x64, 0x5D, 0x19, 0x73],
    [0x60, 0x81, 0x4F, 0xDC, 0x22, 0x2A, 0x90, 0x88, 0x46, 0xEE, 0xB8, 0x14, 0xDE, 0x5E, 0x0B, 0xDB],
    [0xE0, 0x32, 0x3A, 0x0A, 0x49, 0x06, 0x24, 0x5C, 0xC2, 0xD3, 0xAC, 0x62, 0x91, 0x95, 0xE4, 0x79],
    [0xE7, 0xC8, 0x37, 0x6D, 0x8D, 0xD5, 0x4E, 0xA9, 0x6C, 0x56, 0xF4, 0xEA, 0x65, 0x7A, 0xAE, 0x08],
    [0xBA, 0x78, 0x25, 0x2E, 0x1C, 0xA6, 0xB4, 0xC6, 0xE8, 0xDD, 0x74, 0x1F, 0x4B, 0xBD, 0x8B, 0x8A],
    [0x70, 0x3E, 0xB5, 0x66, 0x48, 0x03, 0xF6, 0x0E, 0x61, 0x35, 0x57, 0xB9, 0x86, 0xC1, 0x1D, 0x9E],
    [0xE1, 0xF8, 0x98, 0x11, 0x69, 0xD9, 0x8E, 0x94, 0x9B, 0x1E, 0x87, 0xE9, 0xCE, 0x55, 0x28, 0xDF],
    [0x8C, 0xA1, 0x89, 0x0D, 0xBF, 0xE6, 0x42, 0x68, 0x41, 0x99, 0x2D, 0x0F, 0xB0, 0x54, 0xBB, 0x16]
]

INV_SBOX = [
    [0x52, 0x09, 0x6A, 0xD5, 0x30, 0x36, 0xA5, 0x38, 0xBF, 0x40, 0xA3, 0x9E, 0x81, 0xF3, 0xD7, 0xFB],
    [0x7C, 0xE3, 0x39, 0x82, 0x9B, 0x2F, 0xFF, 0x87, 0x34, 0x8E, 0x43, 0x44, 0xC4, 0xDE, 0xE9, 0xCB],
    [0x54, 0x7B, 0x94, 0x32, 0xA6, 0xC2, 0x23, 0x3D, 0xEE, 0x4C, 0x95, 0x0B, 0x42, 0xFA, 0xC3, 0x4E],
    [0x08, 0x2E, 0xA1, 0x66, 0x28, 0xD9, 0x24, 0xB2, 0x76, 0x5B, 0xA2, 0x49, 0x6D, 0x8B, 0xD1, 0x25],
    [0x72, 0xF8, 0xF6, 0x64, 0x86, 0x68, 0x98, 0x16, 0xD4, 0xA4, 0x5C, 0xCC, 0x5D, 0x65, 0xB6, 0x92],
    [0x6C, 0x70, 0x48, 0x50, 0xFD, 0xED, 0xB9, 0xDA, 0x5E, 0x15, 0x46, 0x57, 0xA7, 0x8D, 0x9D, 0x84],
    [0x90, 0xD8, 0xAB, 0x00, 0x8C, 0xBC, 0xD3, 0x0A, 0xF7, 0xE4, 0x58, 0x05, 0xB8, 0xB3, 0x45, 0x06],
    [0xD0, 0x2C, 0x1E, 0x8F, 0xCA, 0x3F, 0x0F, 0x02, 0xC1, 0xAF, 0xBD, 0x03, 0x01, 0x13, 0x8A, 0x6B],
    [0x3A, 0x91, 0x11, 0x41, 0x4F, 0x67, 0xDC, 0xEA, 0x97, 0xF2, 0xCF, 0xCE, 0xF0, 0xB4, 0xE6, 0x73],
    [0x96, 0xAC, 0x74, 0x22, 0xE7, 0xAD, 0x35, 0x85, 0xE2, 0xF9, 0x37, 0xE8, 0x1C, 0x75, 0xDF, 0x6E],
    [0x47, 0xF1, 0x1A, 0x71, 0x1D, 0x29, 0xC5, 0x89, 0x6F, 0xB7, 0x62, 0x0E, 0xAA, 0x18, 0xBE, 0x1B],
    [0xFC, 0x56, 0x3E, 0x4B, 0xC6, 0xD2, 0x79, 0x20, 0x9A, 0xDB, 0xC0, 0xFE, 0x78, 0xCD, 0x5A, 0xF4],
    [0x1F, 0xDD, 0xA8, 0x33, 0x88, 0x07, 0xC7, 0x31, 0xB1, 0x12, 0x10, 0x59, 0x27, 0x80, 0xEC, 0x5F],
    [0x60, 0x51, 0x7F, 0xA9, 0x19, 0xB5, 0x4A, 0x0D, 0x2D, 0xE5, 0x7A, 0x9F, 0x93, 0xC9, 0x9C, 0xEF],
    [0xA0, 0xE0, 0x3B, 0x4D, 0xAE, 0x2A, 0xF5, 0xB0, 0xC8, 0xEB, 0xBB, 0x3C, 0x83, 0x53, 0x99, 0x61],
    [0x17, 0x2B, 0x04, 0x7E, 0xBA, 0x77, 0xD6, 0x26, 0xE1, 0x69, 0x14, 0x63, 0x55, 0x21, 0x0C, 0x7D],
]

RCON = [
    0x00, 0x01, 0x02, 0x04, 0x08, 0x10, 0x20, 0x40, 0x80, 0x1B, 0x36
]

#### Data Representations

Recall that all data stored in the computer are formatted as bytes. In this assignment, we are going to represent an n-bytes data as a 1-D hexidecimals array. For example, a 5-byte data can be represented as:

In [88]:
data = [0xff, 0x01, 0x4b, 0x22, 0x00]

0x informs the computer that it is a hexadecimal representation, and the following characters (i.e. 4b) represents the value in hexadecimal format. Note that the value for each byte is 0~255, corresponding to 0x00~0xff.

Before we start our implementation, we need some help functions for basic operations.


AES operates on a 4x4 2-D <b>column-major order</b> matrix of 16 bytes. We thus need to convert our 1-D byte array data into such a 4x4 2-D byte matrix. Now finish "to_matrix" and "to_bytes" that convert the 1-D byte array to its 2-D <b>column-major order</b> matrix and convert it back respectively.

![Image](imgs/1.png)

In [89]:
## Input: 1-D byte array
## Output: 2-D byte matrix
def to_matrix(data):
    rows, cols = 4, 4
    index=0
    matrix = [[] for _ in range(rows)]
    for col in range(cols):
        for row in range(rows):
            matrix[row].append(data[index])
            index+=1

    return matrix

## Input: 2-D byte matrix
## Output: 1-D byte array
def to_array(matrix):
    rows, cols = 4, 4
    array=[]
    for col in range(cols):
        for row in range(rows):
            array.append(matrix[row][col])
    return array

In [90]:
## For your check
test_array = [0x01, 0x02, 0x03, 0x04, 0x05, 0x06, 0x07, 0x08, 0x09, 0x0A, 0x0B, 0x0C, 0x0D, 0x0E, 0x0F, 0x10]
assert len(to_matrix(test_array)) == 4
for row in to_matrix(test_array):
    assert len(row) == 4
assert to_array(to_matrix(test_array)) == test_array

AddRoundKey operated on the 4*4 2-D matrix.

![Image](imgs/2.png)

In [91]:
## Input: 2-D byte matrix input block
## Input: 2-D byte matrix key
## Output: 2-D byte matrix output block 
def AddRoundKey(matrix_input, matrix_key):
    matrix_output = [[0 for _ in range(4)] for _ in range(4)]
    for row in range(4):
        for col in range(4):
            matrix_output[row][col]=matrix_input[row][col]^matrix_key[row][col]

    return matrix_output


In [92]:
## For your check
test_array_in = [0x01, 0x02, 0x03, 0x04, 0x05, 0x06, 0x07, 0x08, 0x09, 0x0A, 0x0B, 0x0C, 0x0D, 0x0E, 0x0F, 0x10]
test_array_key = [0x01, 0x02, 0x03, 0x04, 0x05, 0x06, 0x07, 0x08, 0x09, 0x0A, 0x0B, 0x0C, 0x0D, 0x0E, 0x0F, 0x10]
assert AddRoundKey(to_matrix(test_array_in), to_matrix(test_array_key))[2][1] == 0x00


SubBytes and its inverse opertion operated on the 4*4 2-D matrix.


![Image](imgs/3a.png)

In [1]:
## Input: 2-D byte matrix
## Output: 2-D byte matrix
def SubBytes(matrix):
    for row in range(4):
        for col in range(4):
            col1=matrix[row][col]&(15) #lower half
            row1=matrix[row][col]>>4
            matrix[row][col]=SBOX[row1][col1]


    return matrix

## Input: 2-D byte matrix
## Output: 2-D byte matrix
def SubBytes_inv(matrix):
    for row in range(4):
        for col in range(4):
            col1=matrix[row][col]&(15) #lower half
            row1=matrix[row][col]>>4
            matrix[row][col]=INV_SBOX[row1][col1]


    return matrix

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


In [94]:
## For your check
test_array = [0x01, 0x02, 0x03, 0x04, 0x05, 0x06, 0x07, 0x08, 0x09, 0x0A, 0x0B, 0x0C, 0x0D, 0x0E, 0x0F, 0x10]
test_matrix = to_matrix(test_array)
# print(test_matrix)
assert SubBytes_inv(SubBytes(test_matrix)) == test_matrix
# print(test_matrix)


ShiftRows and its inverse opertion operated on the 4*4 2-D matrix.

![Image](imgs/4.png)

In [95]:
## Input: 2-D byte matrix
## Output: 2-D byte matrix
def ShiftRows(matrix):
    ### Your code starts ###
     # Row 1: Left shift by 1
    matrix[1][:] = matrix[1][1:] + matrix[1][:1]
    
    # Row 2: Left shift by 2
    matrix[2][:] = matrix[2][2:] + matrix[2][:2]
    
    # Row 3: Left shift by 3
    matrix[3][:] = matrix[3][3:] + matrix[3][:3]

    ### Your code ends ###
    return matrix

## Input: 2-D byte matrix
## Output: 2-D byte matrix
def ShiftRows_inv(matrix):
    ### Your code starts ###
    # Row 1: Right shift by 1
    matrix[1][:] = matrix[1][-1:] + matrix[1][:-1]
    
    # Row 2: Right shift by 2
    matrix[2][:] = matrix[2][-2:] + matrix[2][:-2]
    
    # Row 3: Right shift by 3
    matrix[3][:] = matrix[3][-3:] + matrix[3][:-3]

    ### Your code ends ###
    return matrix

In [96]:
## For your check
test_array = [0x01, 0x02, 0x03, 0x04, 0x05, 0x06, 0x07, 0x08, 0x09, 0x0A, 0x0B, 0x0C, 0x0D, 0x0E, 0x0F, 0x10]
test_matrix = to_matrix(test_array)
assert ShiftRows_inv(ShiftRows(test_matrix)) == test_matrix


MixColumns and its inverse opertion operated on the 4*4 2-D matrix.

(Hint: the individual additions and multiplications are perfromed in $GF(2^8)$ with irreducible polynomial: $x^8 + x^4 + x^3 + x + 1$ (or 0x11b in hex).)

![Image](imgs/5.png)

In [110]:
## Input: 2-D byte matrix
## Output: 2-D byte matrix

#====Additional Functions Used===
#MUltiplication under GF(2^8)
def GFmul(a, b):
    result = 0
    for _ in range(8):  # since bytes are 8-bit
        if b & 1:
            result ^= a  # add (XOR) current a to result
        carry = a & 0x80  # if MSB is set (bit 7), we’ll overflow
        a <<= 1           # multiply a by x (shift left)
        if carry:
            a ^= 0x11b    # reduce mod m(x)
        b >>= 1           # move to next bit of b
    return result & 0xFF  # return only the last 8 bits
    
# MixColumns transformation on a perticular column
def mix_column(col):
    c0, c1, c2, c3 = col
    return [
        GFmul(c0, 2) ^ GFmul(c1, 3) ^ c2 ^ c3,
        c0 ^ GFmul(c1, 2) ^ GFmul(c2, 3) ^ c3,
        c0 ^ c1 ^ GFmul(c2, 2) ^ GFmul(c3, 3),
        GFmul(c0, 3) ^ c1 ^ c2 ^ GFmul(c3, 2)
    ]

# Inverse MixColumns transformation on a perticular column
def mix_column_inv(col):
    c0, c1, c2, c3 = col
    return [
        GFmul(c0, 14) ^ GFmul(c1, 11) ^ GFmul(c2, 13) ^ GFmul(c3, 9),
        GFmul(c0, 9) ^ GFmul(c1, 14) ^ GFmul(c2, 11) ^ GFmul(c3, 13),
        GFmul(c0, 13) ^ GFmul(c1, 9) ^ GFmul(c2, 14) ^ GFmul(c3, 11),
        GFmul(c0, 11) ^ GFmul(c1, 13) ^ GFmul(c2, 9) ^ GFmul(c3, 14)
    ]


def MixColumns(matrix):
    for i in range(4):
        col = [matrix[j][i] for j in range(4)]
        new_col = mix_column(col)
        for j in range(4):
            matrix[j][i] = new_col[j]

    return matrix

## Input: 2-D byte matrix
## Output: 2-D byte matrix
def MixColumns_inv(matrix):
    for i in range(4):
        col = [matrix[j][i] for j in range(4)]
        new_col = mix_column_inv(col)
        for j in range(4):
            matrix[j][i] = new_col[j]
    return matrix


In [98]:
## For your check
test_array = [0x01, 0x02, 0x03, 0x04, 0x05, 0x06, 0x07, 0x08, 0x09, 0x0A, 0x0B, 0x0C, 0x0D, 0x0E, 0x0F, 0x10]
test_matrix = to_matrix(test_array)
assert MixColumns_inv(MixColumns(test_matrix)) == test_matrix

--------------------------------------

## Section 1: AES-128 Key Scheduling

Tranditional AES applies the Rijndael key schedule to expand the master key to n round keys. Now we are going to implement it.
We use an 1-D byte array to represent the master key, and a 2-D byte array list to represent the entire expanded key.

(master key: [0x11, 0x22 .....])

(expanded key: [[0x11, 0x22 .....], [0x11, 0x22 .....], [0x11, 0x22 .....], ....])

![Image](imgs/6.png)


Before we implement the key expansion, we need the core function to compute the next round key. 
Recall that each round key is computed from the last round key. Finish the 'compute_next_rk' below:

(Hint: Round Constant (RC) has already been defined in Section 0 as 'RCON')

In [99]:
## Input: 1-D byte array for the last round key
## Input: int for the round of computed round key 
##        (say we are computing rk1, then we invoeke this function as compute_next_rk(rk0, 1))
## Output: 1-D byte array for the next round key

def compute_next_rk(current_rk, computed_round):
    rk_matrix = to_matrix(current_rk)
    ### Your code starts ###
    new_matrix = [[0] * 4 for _ in range(4)]
    last_column = [rk_matrix[row][3] for row in range(4)]

    rotated_column = last_column[1:] + last_column[:1]

    substituted_from_SBOX=[]
    for element in rotated_column:
        col1=element&(15)
        row1=element>>4
        substituted_from_SBOX.append(SBOX[row1][col1])
    
    substituted_from_SBOX[0] =substituted_from_SBOX[0] ^ RCON[computed_round]

    #first column : W0^g
    for row in range(4):
        new_matrix[row][0] = rk_matrix[row][0] ^ substituted_from_SBOX[row]
    # Compute rest using: w[i] = w[i-1] ⊕ w[i-4]
    for col in range(1, 4):
        for row in range(4):
            new_matrix[row][col] = new_matrix[row][col - 1] ^ rk_matrix[row][col]    
    
    ### Your code ends ###
    next_rk = to_array(new_matrix)
    return next_rk



The entire key expanison for AES 128. The input is the master key represented as an 1-D byte array. The entire expanded key is a 2-D byte array list, where each row is a round key represented as an 1-D byte array.

In [100]:
## Input: 1-D byte array for the master key
## Output: 2-D byte array list for the entire expanded key
def AES128_key_expansion(master_key):
    expanded_key = []
    expanded_key.append(master_key)
    last_key=0
    for computed_round in range(1,11):
        next_round_key=(compute_next_rk(expanded_key[last_key], computed_round))
        expanded_key.append(next_round_key)
        last_key=last_key+1


    return expanded_key


In [101]:
## For your check
test_master_key = [0x01, 0x02, 0x03, 0x04, 0x05, 0x06, 0x07, 0x08, 0x09, 0x0A, 0x0B, 0x0C, 0x0D, 0x0E, 0x0F, 0x10]
expanded_key = AES128_key_expansion(test_master_key)
for round_key in expanded_key:
    assert len(round_key) == 16

---------------------------------------------------------

## Section 2: AES-128 Encryption and Decryption for one block



the encryption operation. The function take the 1-D byte array master-key and one block 1-D byte array plaintext as inputs, and output the corresponding 1-D byte array ciphertext.

In [102]:
## Input: 1-D byte array for the master key
## Input: 1-D byte array for one block of plaintext
## Output: 1-D byte array for the ciphertext
def AES128_encrypt(master_key, plaintext):
    
    ### Your code starts ###
    key_matrix=to_matrix(master_key)
    plaintext_matrix=to_matrix(plaintext)
    expanded_key= AES128_key_expansion(master_key)
    curr_text=AddRoundKey(plaintext_matrix, to_matrix(expanded_key[0]))

    for roundd in range(1,10):
        curr_text=SubBytes(curr_text)
        curr_text=ShiftRows(curr_text)
        curr_text=MixColumns(curr_text)
        curr_text=AddRoundKey(curr_text, to_matrix(expanded_key[roundd]))
    #last round no mixcol
    curr_text=SubBytes(curr_text)
    curr_text=ShiftRows(curr_text)
    curr_text=AddRoundKey(curr_text,to_matrix(expanded_key[10]))
    ciphertext=to_array(curr_text)
    ### Your code ends ###

    return ciphertext


The decryption operation. The function take the 1-D byte array master-key and one block 1-D byte array ciphertext as inputs, and output the corresponding 1-D byte array plaintext.

In [103]:
## Input: 1-D byte array for the master key
## Input: 1-D byte array for one block of ciphertext
## Output: 1-D byte array for the plaintext
def AES128_decrypt(master_key, ciphertext):
    
    ### Your code starts ###
    key_matrix=to_matrix(master_key)
    ciphertext_matrix=to_matrix(ciphertext)
    expanded_key= AES128_key_expansion(master_key)
    curr_text=AddRoundKey(ciphertext_matrix, to_matrix(expanded_key[10]))

    for roundd in range(9,0,-1):
        curr_text=ShiftRows_inv(curr_text)
        curr_text=SubBytes_inv(curr_text)
        curr_text=AddRoundKey(curr_text, to_matrix(expanded_key[roundd]))
        curr_text=MixColumns_inv(curr_text)

    curr_text=ShiftRows_inv(curr_text)
    curr_text=SubBytes_inv(curr_text)
    curr_text=AddRoundKey(curr_text, to_matrix(expanded_key[0]))

    plaintext=to_array(curr_text)
   
    ### Your code ends ###

    return plaintext

---------------------------------------------------------

## Section 3: AES-128 with Block-Cipher Mode

To encrypt/decrypt long messages, AES should be integrated with block-cipher modes such as CBC, CTR. Futhermore, the length of real-world plaintexts does not always be the exact multiplies of AES block size. As a result, we need pad and unpad our plaintext before encryption/after decryption. 

For simplicity, we consider the minimum unit for a plaintext element is <b>byte</b> in this assignment. In other words, we assume the plaintext is always a sequence of bytes, but not 19 bits/22 bits etc.

Following our assumption, we pad/unpad the plaintext at <b>byte level</b> for our AES-CBC implementation. It follows:
<li> pad: If the last block matches the AES-128 block size already, do nothing. Else, pad the last block with '0x00' until it fullfill the block.</li>
<li> unpad: Start from the last byte, remove all continuous '0x00' until a none-zero byte comes.</li>

We have provided the pad and unpad implementation below. We will make use of them to finish the implementation for AES128_CBC_encrypt and AES128_CBC_decrypt. Both of the encryption and decrytion take an 1-D byte array IV, an 1-D byte array master key, and an 1-D byte array plaintext/ciphertext. The output is the corresponding 1-D byte array ciphertext/plaintext.

In [104]:
## Input: 1-D byte array for the plaintext
## Output: 1-D byte array for the padded plaintext
def pad(plaintext):
    pad_elm = 0x00
    if len(plaintext) % 16 == 0:
        pad_len = 0
    else:
        pad_len = 16 - len(plaintext) % 16
    padded_plaintext = plaintext + [pad_elm] * pad_len
    return padded_plaintext

## Input: 1-D byte array for the padded plaintext
## Output: 1-D byte array for the unpadded plaintext
def unpad(padded_plaintext):
    pad_elm = 0x00
    for i in range(len(padded_plaintext)-1, -1, -1):
        if padded_plaintext[i] != pad_elm:
            break
    unpadded_plaintext = padded_plaintext[:i+1]
    return unpadded_plaintext

In [105]:
## Input: 1-D byte array for the IV (Assume the IV is always 16 bytes)
## Input: 1-D byte array for the master key
## Input: 1-D byte array for the plaintext
## Output: 1-D byte array for the ciphertext
def AES128_CBC_encrypt(iv, master_key, plaintext):
    padded_plaintext = pad(plaintext)
    
    ciphertext=[]
    prev_text=iv

    for block in range(0,len(padded_plaintext),16):
        curr_text=[]
        for i in range(16):
            curr_text.append(padded_plaintext[block+i]^prev_text[i])
        curr_text=AES128_encrypt(master_key, curr_text)
        prev_text=curr_text
        ciphertext.extend(curr_text)


    return ciphertext


## Input: 1-D byte array for the IV (Assume the IV is always 16 bytes)
## Input: 1-D byte array for the master key
## Input: 1-D byte array for the ciphertext
## Output: 1-D byte array for the plaintext
def AES128_CBC_decrypt(iv, master_key, ciphertext):
    
    plaintext=[]
    prev_text=iv

    for block in range(0,len(ciphertext),16):
        curr_text=[]
        xor_text=[]
        curr_text=AES128_decrypt(master_key, ciphertext[block:block+16])
        for i in range(16):
            xor_text.append(prev_text[i]^curr_text[i])
        prev_text=ciphertext[block:block+16]
        plaintext.extend(xor_text)

    unpadded_plaintext = unpad(plaintext)
    return unpadded_plaintext

In [109]:
## For your check
test_plaintext = [0xff] * 22
test_master_key = [0x01, 0x02, 0x03, 0x04, 0x05, 0x06, 0x07, 0x08, 0x09, 0x0A, 0x0B, 0x0C, 0x0D, 0x0E, 0x0F, 0x10]
iv = [0x00] * 16
ciphertext = AES128_CBC_encrypt(iv, test_master_key, test_plaintext)
# print(ciphertext)
decrypted_plaintext = AES128_CBC_decrypt(iv, test_master_key, ciphertext)
# print(decrypted_plaintext)
assert len(ciphertext) == 32
assert len(decrypted_plaintext) == len(unpad(test_plaintext))

---------------------------------------------------------

## Section 4: Discussions

Important questions.

<font color="red" size="4">Question 1:</font> 
A full AES round in AES-128 encryption can be concluded as a sequence of Substitute Bytes, Shift Rows, Mix Columns, and Add Round Key.
However, AES-128 encryption starts with an individual Add Round Key before the first round starts. Explain why such an Add Round Key operation is needed before the first round starts. Or in other words, explain why AES-128 encryption does not start with a full round including all four operations directly.

<span style="color:green;">(Double click this block)</span> Your answer goes there..............................
The initial AddRoundKey operation in AES-128 encryption is crucial for breaking symmetry and introducing immediate key dependence into the cipher. If encryption were to begin with a full round (i.e., starting with SubBytes, ShiftRows, and MixColumns), the early transformations would be purely algebraic and independent of the secret key. This would make the first round act like a fixed permutation, which could be exploited by attackers through known-plaintext or structural attacks.

By applying AddRoundKey before any other operations, AES ensures that the input data is thoroughly mixed with the secret key right from the start. This early key injection disrupts predictable patterns in the input and strengthens diffusion and confusion properties from the very beginning, thereby enhancing the algorithm's overall security.

<font color="red" size="4">Question 2:</font> 

What is the problem for the padding strategy we used in our AES-CBC? Describe it, and describe a solution at the situation where only the ciphertext and its random IV can be transmitted between two parties.

Note that we assume the minumum message unit is <b>byte</b>.

<span style="color:green;">(Double click this block)</span> Your answer goes there..............................

One problem I can see with the zero-byte padding strategy in AES-CBC is that it creates ambiguity when the original message ends with one or more zero bytes. In such cases, the receiver cannot distinguish padding from actual message content after decryption, leading to possible data loss. Since only the ciphertext and IV are transmitted (and not the message length), the padding must be self-descriptive.

From a article I came to know about a secure and effective solution is to use PKCS#7 padding, where each padding byte indicates the number of padding bytes added. This ensures that the receiver can unambiguously and safely remove the padding without needing extra information, preserving both correctness and security.

<font color="red" size="4">Question 3:</font> 

Consider an alternative usage of AES-CBC-encryption. Assume the IV is always fixed as '0x00'. Then, for arbitrary plaintext $m$, Alice use the ciphertext of the last encryption block as the unique $Tag$ for $m$. Assume an adversary can continuous query $Tag_i$ for arbitrary $m_i$ from Alice. With this power, can the adversary generate a valid $Tag_f$ for a message $m_f$ without querying it from Alice? If yes, describe the process to compute $Tag_f$. If no, describe how AES-CBC-encryption prevents such a tag forge.

<span style="color:green;">(Double click this block)</span> Your answer goes there..............................

Yes, the adversary can generate a valid tag for a message mfmf​ without querying it from Alice. This is because using AES-CBC with a fixed IV (such as 0x00) makes the encryption process deterministic and exposes the system to chosen-plaintext attacks.

In CBC mode, the encryption of each block depends on the ciphertext of the previous block, and the first block is XORed with the IV. When the IV is fixed (especially to 0x00), the encryption of the same plaintext block always yields the same ciphertext block. This removes the randomness that normally protects against such attacks.

Here’s how the adversary can exploit this:

    The adversary sends a message mm (say, 1 block) to Alice and receives the tag, which is the last ciphertext block C1=AESk(m)C1​=AESk​(m).

    Now, the adversary constructs a new message by concatenating mm with a new block m2′ of their choice:
    mf=m∣∣m2′

    They already know C1=AESk(m), so they can compute the new tag as:
    C2=AESk(m2′⊕C1)

    Thus, they know the valid ciphertext blocks for the message mf, and in particular, they know the last ciphertext block, which serves as the tag.

    This process can be extended — using known ciphertexts, the adversary can construct valid tags for longer messages of the form m∣∣m2′∣∣m3′∣∣… without needing to ask Alice to encrypt those messages.

This attack shows that using the last block of AES-CBC encryption as a tag does not provide message integrity or authentication when the IV is fixed. The attacker can manipulate and forge valid ciphertext-tag pairs, defeating the purpose of a Message Authentication Code (MAC).


--------------------------------------------

End of this Project